In [23]:
#import libraries.

import cv2
import numpy as np
from collections import deque

In [24]:
#define red_upper and red_lower(will depend on whatever colour we are using)
red_upper = np.array([150,150,50])
red_lower = np.array([180,255,150])

#define blackboard for writing.
blackboard = np.zeros((512,512,3), np.uint8)

#define deque points for storing whatever we will be drawing on real time.
points = deque(maxlen = 1024)

#define kernal for  doing cnn stuff for erosion and dilation.
kernel = np.ones((5,5), np.uint8)

In [ ]:
#start loop
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    #define hsv and gray and frame
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    #define red and remove bg
    red = cv2.inRange(hsv, red_upper, red_lower)
    #remove noises
    red_median = cv2.medianBlur(red, 15)

    #find contours
    contours, _ = cv2.findContours(red_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #find contour of max area
    if len(contours) >  0:
        contour_list = sorted(contours, key = cv2.contourArea, reverse = True)
        contour = contour_list[0] #contour of  max area
        
        #find centroid ofcontour
        M = cv2.moments(contour) #M is dict
        centroid = ((M["m10"]/M["m00"]), (M["m01"]/M["m00"]))

        #keep storing these points in deque fro left
        points.appendleft(centroid)
        
        #(optional) draw letter on frame
        ((x,y), radius) = cv2.minEnclosingCircle(contour)
        cv2.circle(frame, (int(x),int(y)), int(radius), (0,255,0), 2)
        
        #when we can't find any contour, it means that we are done with writting letter. And thus we should now 
        #connect those points and remove noise 
    
    elif len(contours) == 0:
        if len(points) != 0:
            for i in range(1, len(points)):
                cv2.line(blackboard, points[i], points[i-1], (255,255,255), 10)
                cv2.line(frame, points[i], points[i-1], (0,0,0), 2)
                #remove noise
                blackboard_median = cv2.medianBlur(blackboard, 15)
                
                #find contour that encloses the letter
                _, alphabet_contours, _ = cv2.findContour(blackboard_median, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #if we have multiple contours
                if len(alphabet_contours) >= 1:
                    alphabet_contour_list = sorted(alphabet_contours, key = cv2.contourArea, reverse =True)
                    alphabet_contour = alphabet_contour_list[0]
                    
                    #enclose that letter in square of min area.
                    x, y, w, h = cv2.boundingRect(alphabet_contour)
                    alphabet = blackboard[y-10:y+h+10, x-10:x+w+10]
                    alphabet_gray = cv2.cvtColor(alphabet, cv2.COLOR_BGR2GRAY)
                    
                    #reshape it to (28,28)
                    resized_alphabet = cv2.resize(alphabet_gray, (28,28))
                    #all pixel values should be in between 0 to 1
                    resized_alphabet = resized_alphabet/255

                    #give resized_alphabet to our model.
                    #...........
                    
                    #write prediction at bottom.
                    #cv2.putText(frame, f"Prediction: {...}", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 5)


                    #add sound stuff.
                    #..........
                    
                    #show
                    cv2.imshow("frame", frame)
                    cv2.imshow("red_median", red_median)
                    cv2.imshow("blackboard_median", blackboard_median)
                    

    #clear deque and blackboard for next trail.
    blackboard = np.zeros((512,512), np.uint8)
    points = deque(maxlen = 1024)
    
    #close loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()